## Univariate and Multivariate Modeling and best fit - BASEBALL

### Create univariate linear regression to predict player salary

In [2]:
import pandas
import numpy as np
# from scipy.stats import 
import matplotlib.pyplot as plt

PLAYER_SALARY = './baseball_data/Salaries.csv'
salary = pandas.read_csv(PLAYER_SALARY)
salary.head(4)

,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
